In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
# from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 데이터 로드 및 전처리
df = pd.read_excel('/content/drive/MyDrive/졸작/data/최종 사용 데이터/최종_빈도+소비_분석준비_0928.xlsx')
df = df.drop('Unnamed: 0', axis=1)
df

,일자,C1,C2,C3,C4,C5,C6,C7,M1,M2,M3,M4,M5,M6,M7,expenditure_tour
0,20221029,8,1,0,3,0,0,0,0,0,12,0,0,0,0,47794532000
1,20221030,1289,326,152,422,64,61,3,226,3,1560,295,193,7,33,37155395000
2,20221031,1380,516,273,189,215,120,6,486,12,1711,294,159,27,10,29237326000
3,20221101,1037,359,174,117,124,101,5,321,7,1106,241,178,55,9,29130528000
4,20221102,1077,319,189,57,109,59,19,301,0,1175,197,71,82,3,30257440000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,20231124,8,5,4,3,0,2,1,0,0,23,0,0,0,0,37160819000
366,20231126,4,6,0,0,0,2,0,0,0,6,6,0,0,0,37049141000
367,20231128,13,10,1,0,2,3,3,0,0,31,1,0,0,0,29963826000
368,20231129,1,0,1,0,0,0,0,0,0,2,0,0,0,0,31243906000


In [4]:
# '일자'열 datetime으로 타입변경
df['date'] = pd.to_datetime(df['일자'].astype(str), format='%Y%m%d')

# df.info()

In [5]:
# 날짜 확인
print(df['date'].unique())

# 결측치 확인
print(df.isnull().sum())

<DatetimeArray>
['2022-10-29 00:00:00', '2022-10-30 00:00:00', '2022-10-31 00:00:00',
 '2022-11-01 00:00:00', '2022-11-02 00:00:00', '2022-11-03 00:00:00',
 '2022-11-04 00:00:00', '2022-11-05 00:00:00', '2022-11-06 00:00:00',
 '2022-11-07 00:00:00',
 ...
 '2023-11-18 00:00:00', '2023-11-19 00:00:00', '2023-11-21 00:00:00',
 '2023-11-22 00:00:00', '2023-11-23 00:00:00', '2023-11-24 00:00:00',
 '2023-11-26 00:00:00', '2023-11-28 00:00:00', '2023-11-29 00:00:00',
 '2023-11-30 00:00:00']
Length: 370, dtype: datetime64[ns]
일자                  0
C1                  0
C2                  0
C3                  0
C4                  0
C5                  0
C6                  0
C7                  0
M1                  0
M2                  0
M3                  0
M4                  0
M5                  0
M6                  0
M7                  0
expenditure_tour    0
date                0
dtype: int64


In [6]:
# 날짜를 인덱스로
df.set_index(keys=['date'], inplace=True)
df.head()

,일자,C1,C2,C3,C4,C5,C6,C7,M1,M2,M3,M4,M5,M6,M7,expenditure_tour
date,,,,,,,,,,,,,,,,
2022-10-29,20221029,8,1,0,3,0,0,0,0,0,12,0,0,0,0,47794532000
2022-10-30,20221030,1289,326,152,422,64,61,3,226,3,1560,295,193,7,33,37155395000
2022-10-31,20221031,1380,516,273,189,215,120,6,486,12,1711,294,159,27,10,29237326000
2022-11-01,20221101,1037,359,174,117,124,101,5,321,7,1106,241,178,55,9,29130528000
2022-11-02,20221102,1077,319,189,57,109,59,19,301,0,1175,197,71,82,3,30257440000


In [7]:
# 독립변수와 종속변수 정의
# X = df[['정부', '정치 및 사회 조직', '문화 및 언론', '의료', '교육 및 연구 기관',
#         '경제 및 민간', '국제 및 외교', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7']]

X = df[['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7']]

# X = df.drop(df[['expenditure_tour', '일자']], axis=1)
# X = df[['C4', 'C7', 'M1', 'M6']]
y = df['expenditure_tour']

# Z-점수 표준화 - 독립변수만 진행(y는 안함)
# scaler = StandardScaler()  # 변수명을 scaler로 수정
# X_standardized = scaler.fit_transform(X)
# X_standardized_df = pd.DataFrame(X_standardized, columns=X.columns, index=X.index)
# X_standardized_df = X
# # X_standardized_df['date'] = df['일자']
# X_standardized_df.head()

In [10]:
X.shape

(370, 14)

In [11]:
# 훈련/검증 데이터셋 생성.
# 데이터 길이 확인
n = len(X)

# 80%까지를 훈련 데이터로 사용
train_size = int(n * 0.8)
train_size

# 시간 순서대로 데이터 분할
X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]

y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]


In [12]:
print(X_train.shape)
print(X_standardized_df_test.shape)

(296, 14)


NameError: name 'X_standardized_df_test' is not defined

## ARIMAX

In [13]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
# !pip install statsmodels==0.14.0
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.arima.model import ARIMAX
# from statsmodels.tsa.arima_model import ARIMA
# from statsmodels.tsa.arima_model import ARIMAX
from sklearn.metrics import mean_squared_error

In [14]:
# 파라미터 범위 설정
p = d = q = range(0, 5)  # 0부터 4까지의 정수
pdq = list(itertools.product(p, d, q))

best_aic = float('inf')  # 초기값 설정
best_param = None

# 모든 조합 시도
for param in pdq:
    try:
        model = SARIMAX(y_train, exog=X_train, order=param)
        model_fit = model.fit()
        if model_fit.aic < best_aic:  # AIC 값이 더 낮으면 갱신
            best_aic = model_fit.aic
            best_param = param
    except:
        continue

print(f'Best ARIMAX parameters: {best_param} with AIC: {best_aic}')


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


Best ARIMAX parameters: (4, 1, 4) with AIC: 14055.79656285467


In [15]:
# 모델 적합
final_model = SARIMAX(y_train, exog=X_train, order=best_param)
final_model_fit = final_model.fit()

# 모델 계수 확인
print(final_model_fit.summary())
print('계수: ', final_model_fit.params)

# 테스트 데이터 예측
# exog_test = test[['external_variable1', 'external_variable2']]  # 테스트 데이터의 외생변수
forecast = final_model_fit.forecast(steps=len(X_test), exog=X_test)


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                               SARIMAX Results                                
Dep. Variable:       expenditure_tour   No. Observations:                  296
Model:               SARIMAX(4, 1, 4)   Log Likelihood               -7004.898
Date:                Tue, 22 Oct 2024   AIC                          14055.797
Time:                        11:03:57   BIC                          14140.597
Sample:                             0   HQIC                         14089.753
                                - 296                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
C1         -1.211e+07   2.69e-09  -4.51e+15      0.000   -1.21e+07   -1.21e+07
C2         -2.178e+07   4.39e-10  -4.96e+16      0.000   -2.18e+07   -2.18e+07
C3         -8.145e+07   2.42e-10  -3.37e+17      0.0

/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [16]:
forecast.index = y_test.index
forecast

,predicted_mean
date,
2023-09-05,3.254997e+10
2023-09-06,3.690631e+10
2023-09-07,3.702437e+10
2023-09-08,3.809166e+10
2023-09-09,3.692938e+10
...,...
2023-11-24,3.469670e+10
2023-11-26,3.468287e+10
2023-11-28,3.480887e+10


In [17]:
y_test

,expenditure_tour
date,
2023-09-05,28908998000
2023-09-06,30300367000
2023-09-07,31405400000
2023-09-08,36583063000
2023-09-09,45219490000
...,...
2023-11-24,37160819000
2023-11-26,37049141000
2023-11-28,29963826000


In [18]:
# RMSE 계산
rmse = np.sqrt(mean_squared_error(y_test, forecast))
print('The RMSE of ARIMAX is:', rmse)


The RMSE of ARIMAX is: 5579452983.504504


In [19]:
# MAPE (Mean Absolute Percentage Error) 계산
mape = np.mean(np.abs((y_test - forecast) / y_test)) * 100
print('The MAPE of ARIMAX is:', mape, '%')


The MAPE of ARIMAX is: 14.838533077184243 %


In [20]:
jjh = pd.DataFrame({'Predicted' : forecast, 'Actual' : y_test})
jjh.to_excel('ARIMAX_res.xlsx')